## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "hzhou" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-3-135-190-193.us-east-2.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-3-135-190-193.us-east-2.compute.amazonaws.com:5000'


In [3]:
from mlflow.tracking import MlflowClient

client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [4]:
from mlflow.entities import ViewType
active_experiments = client.search_experiments(view_type=ViewType.ACTIVE_ONLY)

for exp in active_experiments:
    print(f"Name: {exp.name}, ID: {exp.experiment_id}, Artifact Location: {exp.artifact_location}")
 # list_experiments API has been removed, you can use search_experiments instead.()

Name: my-experiment-1, ID: 1, Artifact Location: s3://mlflow-artifact-remote-hzhou/1
Name: Default, ID: 0, Artifact Location: s3://mlflow-artifact-remote-hzhou/0


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/04/04 23:17:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run sneaky-vole-178 at: http://ec2-3-135-190-193.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/5829e345341b4fe2937691cc1f2efbbd
🧪 View experiment at: http://ec2-3-135-190-193.us-east-2.compute.amazonaws.com:5000/#/experiments/1


ProfileNotFound: The config profile (hzhou) could not be found

In [ ]:
mlflow.search_experiments()

### Interacting with the model registry

In [ ]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [ ]:
client.list_registered_models()

In [ ]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)